In [1]:
import numpy as np

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [3]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [4]:
def calc_logloss_modify(y, y_pred):
    a = y * np.log(y_pred) if y_pred != 0 else -np.inf
    b = (1.0 - y) * np.log(1.0 - y_pred) if y_pred != 1 else -np.inf
    err = - np.mean(a + b)
    return err

In [5]:
calc_logloss_modify(0, 0)

inf

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [6]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
def eval_model(X, y, iterations, alpha=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0 and log:
            print(i, W, err)
    return W

In [8]:
W = eval_model(X, y, 2000, alpha=1e-6)
W

200 [ 0.49661415 -0.1385243   0.55568854  1.52288986] nan
400 [ 0.49651415 -0.1387843   0.46368854  1.52274986] nan
600 [ 0.49641415 -0.1390443   0.37168854  1.52260986] nan
800 [ 0.49631415 -0.1393043   0.27968854  1.52246986] nan
1000 [ 0.49621415 -0.1395643   0.18768854  1.52232986] nan
1200 [ 0.49611415 -0.1398243   0.09568854  1.52218986] nan
1400 [ 0.49601419 -0.14008427  0.00370585  1.52204989] 3.05285060704639
1600 [ 0.49599251 -0.14009064 -0.00243037  1.52204931] 0.6047441828966463
1800 [ 0.49597611 -0.1400782  -0.00243041  1.52205868] 0.6047416240148095
2000 [ 0.4959597  -0.14006576 -0.00243045  1.52206805] 0.6047390654109192


C:\Users\GruXsqK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\GruXsqK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  


array([ 0.4959597 , -0.14006576, -0.00243045,  1.52206805])

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [9]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(W @ X)
    return y_pred_proba

In [10]:
calc_pred_proba(W, X)

array([0.65987503, 0.54404739, 0.8080094 , 0.60340964, 0.40029095,
       0.51840875, 0.67176111, 0.23158333, 0.68659747, 0.69625237])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [11]:
def calc_pred(W, X):
    
    THRESHOLD = 0.5
    m = X.shape[1]
    
    y_pred = np.zeros((1, m))
    W = W.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(W.T, X))

    for i in range(A.shape[1]):
        if (A[:,i] > THRESHOLD): 
            y_pred[:, i] = 1
        elif (A[:,i] <= THRESHOLD):
            y_pred[:, i] = 0
    
    return y_pred

In [12]:
calc_pred(W, X)

array([[1., 1., 1., 1., 0., 1., 1., 0., 1., 1.]])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [13]:
def get_all_score(W, X):
    
    y_pred = calc_pred(W, X)[0]
    accuracy = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
    
    conf_matrix = np.array([[0, 0], [0, 0]])
    
    a = y_pred.astype(np.int32)
    b = y.astype(np.int32)
    
    conf_matrix[0][0] = np.sum(a & b)    # TP 
    conf_matrix[0][1] = np.sum(a & ~b)    # FP
    conf_matrix[1][0] = np.sum(~a & b)    # FN
    conf_matrix[1][1] = np.sum(~a & ~b + 2)    # TN
    
    precision = conf_matrix[0][0]/(conf_matrix[0][0] + conf_matrix[0][1])
    recall = conf_matrix[0][0]/(conf_matrix[0][0] + conf_matrix[1][0])
    f1 = (2 * precision * recall)/(precision + recall)

    return accuracy, conf_matrix, precision, recall, f1

In [14]:
get_all_score(W, X)

(50.0, array([[4, 4],
        [1, 1]]), 0.5, 0.8, 0.6153846153846154)

6. Могла ли модель переобучиться? Почему?

Да, наблюдений очень мало и есть выбросы в данных.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [15]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_/2 * np.sign(np.exp(-W)/(1 + np.exp(-W)**2)))
        if i % (iterations / 10) == 0 and log:
            print(i, W, err)
    return W

In [16]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-4, log=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * (np.exp(-W)/(1 + np.exp(-W)**3)))
        if i % (iterations / 10) == 0 and log:
            print(i, W, err)
    return W

In [17]:
eval_model_l1(X, y, 4000)

400 [ 0.49439234 -0.14215137  0.04031367  1.52574855] 3.8896486137330997
800 [ 0.49265386 -0.14436874  0.0406428   1.52950523] 2.1802208219649595
1200 [ 0.49091161 -0.14658617  0.04053795  1.53325989] 2.1349530464924085
1600 [ 0.48916899 -0.14880267  0.04092275  1.5370152 ] 2.3377243547584357
2000 [ 0.48741429 -0.15103448  0.03351131  1.54075452] 1.1764958946815134
2400 [ 0.48567789 -0.15323661  0.04055299  1.54452184] 2.1427219223662517
2800 [ 0.48393258 -0.1554545   0.03982861  1.54827428] 4.221350807962076
3200 [ 0.48219008 -0.15767464  0.03792715  1.55202562] 1.5673947331399944
3600 [ 0.48044739 -0.15988935  0.03821504  1.55578006] 5.176623245294346
4000 [ 0.47869863 -0.16210626  0.03730217  1.55953072] 1.4911927511055734


C:\Users\GruXsqK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\GruXsqK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  


array([ 0.47869863, -0.16210626,  0.03730217,  1.55953072])

In [18]:
eval_model_l2(X, y, 4000)

C:\Users\GruXsqK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\GruXsqK\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in multiply
  


400 [ 0.4943914  -0.1421497   0.04097981  1.52575044] 3.307573472089633
800 [ 0.49264817 -0.14436716  0.04092868  1.52950616] 2.3406725289836876
1200 [ 0.49089863 -0.14658666  0.03953711  1.53325771] 4.4060028249301215
1600 [ 0.48914847 -0.14880415  0.03925039  1.53701086] 4.580699136794995
2000 [ 0.4873946  -0.15102678  0.03621428  1.54075844] 1.3777888540921535
2400 [ 0.48565901 -0.15324431  0.03566807  1.54451526] 1.330136340063182
2800 [ 0.48392488 -0.1554501   0.041102    1.54828201] 3.143852160733386
3200 [ 0.482187   -0.15766669  0.04082495  1.55203841] 3.4695620601010804
3600 [ 0.48044519 -0.15988155  0.0411382   1.55579442] 2.5484960548369626
4000 [ 0.4786978  -0.16211094  0.03453279  1.559538  ] 1.2444344873871678


array([ 0.4786978 , -0.16211094,  0.03453279,  1.559538  ])